In [1]:
!pip install category-encoders==2.4.0 
!pip install refractml==1.0.3

     |████████████████████████████████| 92kB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 235kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 11.1MB 14.6MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 73.3MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 76.8MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 45.4MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 99.3MB/s eta 0:00:01
     |████████████████████████████████| 307kB 84.8MB/s eta 0:00:01
     |████████████████████████████████| 348kB 85.0MB/s eta 0:00:01
     |████████████████████████████████| 256kB 88.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 77.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 12.0MB/s eta 0:00:01
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.7.3 which is incompatible.
ERROR: jupyterlab-server 2.25.3 has requirement babel>=2.10, but you'll ha

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# !pip uninstall -y urllib3

In [ ]:
# !pip install urllib3==1.26.15

In [ ]:
# !pip show cloudpickle

In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from refractml import *
from refractml.constants import MLModelFlavours

<ipython-input-1-6b562fd3ff42>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
!pip install pandas --target /packages/Python-3.8/23616da5-c36c-4994-8414-2a19942bd81a/3.8

  Using cached https://files.pythonhosted.org/packages/f8/7f/5b047effafbdd34e52c9e2d7e44f729a0655efafb22198c45cf692cdc157/pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/98/5d/5738903efe0ecb73e51eb44feafba32bdba2081263d40c5043568ff60faf/numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py

In [ ]:
import pandas as pd

In [2]:
cars = pd.read_csv("car_new (1).csv")
cars

buying_num  maintenance_num  doors_num  persons_num  lugboot_num  \
0              3                3          0            0            2   
1              3                3          0            0            2   
2              3                3          0            0            1   
3              3                3          0            0            1   
4              3                3          0            0            1   
...          ...              ...        ...          ...          ...   
1722           1                1          3            2            1   
1723           1                1          3            2            1   
1724           1                1          3            2            0   
1725           1                1          3            2            0   
1726           1                1          3            2            0   

      safety_num  class_num  
0              2          2  
1              0          2  
2              1          2  
3              2          2  
4              0          2  
...          ...        ...  
1722           2          1  
1723           0          3  
1724           1          2  
1725           2          1  
1726           0          3  

[1727 rows x 7 columns]

In [3]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [4]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [5]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [6]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [7]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'buying_num': {'type': 'integer', 'format': 'int64'}, 'maintenance_num': {'type': 'integer', 'format': 'int64'}, 'doors_num': {'type': 'integer', 'format': 'int64'}, 'persons_num': {'type': 'integer', 'format': 'int64'}, 'lugboot_num': {'type': 'integer', 'format': 'int64'}, 'safety_num': {'type': 'integer', 'format': 'int64'}}}, 'example': [{'buying_num': 1, 'maintenance_num': 2, 'doors_num': 0, 'persons_num': 2, 'lugboot_num': 0, 'safety_num': 1}, {'buying_num': 3, 'maintenance_num': 2, 'doors_num': 2, 'persons_num': 2, 'lugboot_num': 2, 'safety_num': 2}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [2]}}


In [8]:
!pip freeze | grep -i scikit

scikit-learn==1.4.1.post1


In [9]:
register_model(model, 
               score, 
               name='CarEval_3_10_22Feb', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=True, 
               kyd_score = False
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
